## Work
1. 請比較使用 l1, l1_l2 及不同比例下的訓練結果

In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.regularizers import l1, l2, l1_l2

"""
建立神經網路，並加入 L1 或 L2
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128], l2_ratio=1e-4):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1), 
                                   kernel_regularizer=l2(l2_ratio))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1),
                                   kernel_regularizer=l2(l2_ratio))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 256
MOMENTUM = 0.95
L2_EXP = [1e-2, 1e-4, 1e-8, 1e-12]

In [7]:
results = {}
"""
使用迴圈建立不同的帶不同 L1/L2 的模型並訓練
"""
for regulizer_ratio in L2_EXP:
    keras.backend.clear_session() # 把舊的 Graph 清掉
    print("Experiment with Regulizer = %.6f" % (regulizer_ratio))
    model = build_mlp(input_shape=x_train.shape[1:], l2_ratio=regulizer_ratio)
    model.summary()
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True)
    
    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["acc"]
    valid_acc = model.history.history["val_acc"]
    
    exp_name_tag = "exp-l2-%s" % str(regulizer_ratio)
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}


Experiment with Regulizer = 0.010000
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [=====================

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 3s 52us/step - loss: 2.1585 - acc: 0.2763 - val_loss: 1.9959 - val_acc: 0.3458
Epoch 2/50
50000/50000 [==============================] - 2s 41us/step - loss: 1.9375 - acc: 0.3672 - val_loss: 1.8910 - val_acc: 0.3835
Epoch 3/50
50000/50000 [==============================] - 2s 43us/step - loss: 1.8551 - acc: 0.3982 - val_loss: 1.8306 - val_acc: 0.4050
Epoch 4/50
50000/50000 [==============================] - 2s 43us/step - loss: 1.7974 - acc: 0.4197 - val_loss: 1.7725 - val_acc: 0.4311
Epoch 5/50
50000/50000 [==============================] - 2s 42us/step - loss: 1.7518 - acc: 0.4348 - val_loss: 1.7340 - val_acc: 0.4348
Epoch 6/50
50000/50000 [==============================] - 2s 43us/step - loss: 1.7130 - acc: 0.4491 - val_loss: 1.7004 - val_acc: 0.4498
Epoch 7/50
50000/50000 [==============================] - 2s 42us/step - loss: 1.6778 - acc: 0.4603 - val_loss: 1.6798 - val_acc

50000/50000 [==============================] - 2s 41us/step - loss: 1.8073 - acc: 0.3700 - val_loss: 1.7485 - val_acc: 0.3907
Epoch 3/50
50000/50000 [==============================] - 2s 42us/step - loss: 1.7191 - acc: 0.4003 - val_loss: 1.6865 - val_acc: 0.4019
Epoch 4/50
50000/50000 [==============================] - 2s 42us/step - loss: 1.6573 - acc: 0.4230 - val_loss: 1.6395 - val_acc: 0.4264
Epoch 5/50
50000/50000 [==============================] - 2s 42us/step - loss: 1.6095 - acc: 0.4388 - val_loss: 1.5937 - val_acc: 0.4455
Epoch 6/50
50000/50000 [==============================] - 2s 42us/step - loss: 1.5706 - acc: 0.4499 - val_loss: 1.5690 - val_acc: 0.4505
Epoch 7/50
50000/50000 [==============================] - 2s 42us/step - loss: 1.5378 - acc: 0.4618 - val_loss: 1.5689 - val_acc: 0.4432
Epoch 8/50
50000/50000 [==============================] - 2s 41us/step - loss: 1.5090 - acc: 0.4713 - val_loss: 1.5322 - val_acc: 0.4627
Epoch 9/50
50000/50000 [============================

50000/50000 [==============================] - 2s 42us/step - loss: 1.7184 - acc: 0.3982 - val_loss: 1.6800 - val_acc: 0.4124
Epoch 4/50
50000/50000 [==============================] - 2s 41us/step - loss: 1.6596 - acc: 0.4195 - val_loss: 1.6336 - val_acc: 0.4293
Epoch 5/50
50000/50000 [==============================] - 2s 41us/step - loss: 1.6122 - acc: 0.4364 - val_loss: 1.5971 - val_acc: 0.4378
Epoch 6/50
50000/50000 [==============================] - 2s 41us/step - loss: 1.5740 - acc: 0.4499 - val_loss: 1.5612 - val_acc: 0.4454
Epoch 7/50
50000/50000 [==============================] - 2s 40us/step - loss: 1.5406 - acc: 0.4609 - val_loss: 1.5435 - val_acc: 0.4576
Epoch 8/50
50000/50000 [==============================] - 2s 41us/step - loss: 1.5110 - acc: 0.4711 - val_loss: 1.5235 - val_acc: 0.4633
Epoch 9/50
50000/50000 [==============================] - 2s 41us/step - loss: 1.4846 - acc: 0.4808 - val_loss: 1.5109 - val_acc: 0.4647
Epoch 10/50
50000/50000 [===========================

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
color_bar = ["r", "g", "b", "y", "m", "k"]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.ylim([0, 5])
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()